In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

class Net(nn.Module): 
    def __init__(self, debug=False):
        super(Net, self).__init__() 
        self.l1 = nn.Linear(in_features=10, out_features=64)
        self.l2 = nn.Linear(64, 32)
        self.l3 = nn.Linear(32, 1) 

    def forward(self, x): 
        x = self.l1(x)      # x.shape (n, 10) --> (n, 64)  
        x = F.relu(x)  
        x = self.l2(x)      # x.shape (n, 64) --> (n, 32)  
        x = F.relu(x) 
        x = self.l3(x)      # x.shape (n, 32) --> (n,  1) 
        return x 

net = Net()
print(net.l2.weight.shape, net.l2.bias.shape)
# 创建输入测试ANN
input_data = torch.randn(2, 10)  # 假设输入维度为10

# 前向传播
outputs = net(input_data)
print(outputs) 

torch.Size([32, 64]) torch.Size([32])
tensor([[ 0.0046],
        [-0.0085]], grad_fn=<AddmmBackward0>)


In [2]:
import random, numpy as np
random.seed(123)
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import TensorDataset, Subset, DataLoader

enc = OneHotEncoder(handle_unknown='ignore')  
enc.fit([[11], [12], [13], [14], [15], [16], [17], [18], [19], [20]])
_labels    = [[random.randint(11, 20)] for _ in range(10000)]
_features = enc.transform(_labels).toarray()

sss = StratifiedShuffleSplit(test_size=0.2, random_state=123)  
train_idx, test_idx = next(sss.split(_features, np.array(_labels).flatten())) 
sss = StratifiedShuffleSplit(test_size=0.5, random_state=123) 
valid_idx, test_idx = next(sss.split(_features[test_idx], np.array(_labels).flatten()[test_idx])) 

##################### test purpose ########################## 
c = _features[train_idx].argmax(axis=1) 
print([(i, len([j for j in c if j==i])) for i in range(10)])

c = _features[valid_idx].argmax(axis=1) 
print([(i, len([j for j in c if j==i])) for i in range(10)])

c = _features[test_idx].argmax(axis=1) 
print([(i, len([j for j in c if j==i])) for i in range(10)]) 

print(len(train_idx), len(valid_idx), len(test_idx)) 
#############################################################

features  = torch.tensor(_features, dtype=torch.float32)
labels     = torch.tensor(_labels   , dtype=torch.float32)
dataset = TensorDataset(features, labels)

trainingset = Subset(dataset, train_idx)
trainloader = DataLoader(trainingset, batch_size=64, shuffle=True) 

validset = Subset(dataset, valid_idx)
validloader = DataLoader(validset, batch_size=64, shuffle=True) 

testset = Subset(dataset, test_idx)
testloader = DataLoader(testset, batch_size=64, shuffle=True) 

##################### test purpose ############################ 
for i, (features, labels) in enumerate(testloader,0):
    print(f'testloader第{i}次吐出{len(labels)}组数据')
print(f'testloader总共已吐出{len(testset.dataset)}组数据')
print(f'最后一组吐出的{len(labels)}组数据是:',features, '\n', labels)
###############################################################

[(0, 823), (1, 824), (2, 785), (3, 783), (4, 790), (5, 806), (6, 765), (7, 834), (8, 790), (9, 800)]
[(0, 93), (1, 100), (2, 116), (3, 95), (4, 90), (5, 109), (6, 111), (7, 99), (8, 105), (9, 82)]
[(0, 100), (1, 99), (2, 99), (3, 117), (4, 92), (5, 111), (6, 90), (7, 97), (8, 111), (9, 84)]
8000 1000 1000
testloader第0次吐出64组数据
testloader第1次吐出64组数据
testloader第2次吐出64组数据
testloader第3次吐出64组数据
testloader第4次吐出64组数据
testloader第5次吐出64组数据
testloader第6次吐出64组数据
testloader第7次吐出64组数据
testloader第8次吐出64组数据
testloader第9次吐出64组数据
testloader第10次吐出64组数据
testloader第11次吐出64组数据
testloader第12次吐出64组数据
testloader第13次吐出64组数据
testloader第14次吐出64组数据
testloader第15次吐出40组数据
testloader总共已吐出10000组数据
最后一组吐出的40组数据是: tensor([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0.,

In [3]:
import torch.optim as optim

criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
 
# Training with Validation
epochs = 3
min_valid_loss = np.inf
 
for epoch in range(epochs):
    train_loss = 0.0
    net.train()
    for i, (features, labels) in enumerate(trainloader, start=0): 
        outputs = net(features)
        loss = criterion(outputs,labels)  
        optimizer.zero_grad()
        loss.backward() 
        optimizer.step() 
        train_loss += loss.item()
        if i % 20 == 0:    # print every 20 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] Training Loss: {train_loss / 20:.3f}')
            train_loss = 0.0
     
    valid_loss = 0.0
    net.eval()     # Optional when not using Model Specific layer
    with torch.no_grad():
        for _, (features, labels) in enumerate(validloader, start=0):   
            outputs = net(features) 
            loss = criterion(outputs,labels) 
            valid_loss += loss.item()
        print(f'[{epoch + 1}] Validation Loss: {valid_loss / len(validloader.dataset)}')
     
    if min_valid_loss > valid_loss:
        print(f'Validation Loss Decreased ({min_valid_loss:.6f}--->{valid_loss:.6f})')
        min_valid_loss = valid_loss
        torch.save(net.state_dict(), 'best_model.pth')
        print(f'Better Model Saved as best_model.pth')

[1,     1] Training Loss: 12.821
[1,    21] Training Loss: 158.067
[1,    41] Training Loss: 18.952
[1,    61] Training Loss: 2.091
[1,    81] Training Loss: 0.222
[1,   101] Training Loss: 0.028
[1,   121] Training Loss: 0.004
[1] Validation Loss: 1.6211285372264683e-05
Validation Loss Decreased (inf--->0.016211)
Better Model Saved as best_model.pth
[2,     1] Training Loss: 0.000
[2,    21] Training Loss: 0.001
[2,    41] Training Loss: 0.000
[2,    61] Training Loss: 0.000
[2,    81] Training Loss: 0.000
[2,   101] Training Loss: 0.000
[2,   121] Training Loss: 0.000
[2] Validation Loss: 7.190664882728016e-08
Validation Loss Decreased (0.016211--->0.000072)
Better Model Saved as best_model.pth
[3,     1] Training Loss: 0.000
[3,    21] Training Loss: 0.000
[3,    41] Training Loss: 0.000
[3,    61] Training Loss: 0.000
[3,    81] Training Loss: 0.000
[3,   101] Training Loss: 0.000
[3,   121] Training Loss: 0.000
[3] Validation Loss: 3.1739672579078617e-10
Validation Loss Decreased 

In [4]:
with torch.no_grad():
    correct = 0 
    for _, (features, labels) in enumerate(testloader, start=0):  
        outputs = net(features) 
        correct += sum(np.isclose(outputs,labels,atol=1e-3).flatten())
    print(f'Accuracy of the network on the {len(testloader.dataset)}\
 samples: {100 * correct // len(testloader.dataset)} %')

Accuracy of the network on the 1000 samples: 100 %
